In [42]:
%matplotlib inline
import matplotlib.pyplot as plt
import chainer
import chainer.functions as F
from chainer import Variable
import numpy as np
import sys
sys.path.append('src')
sys.path.append('models')
import importlib
import dataset
import net
importlib.reload(dataset)
from chainer import serializers
import cv2
import os
from progressbar import ProgressBar
import math

In [45]:
def color_jet(x):
    if x < 0.25:
        b = 255
        g = x / 0.25 * 255
        r = 0
    elif x >= 0.25 and x < 0.5:
        b = 255 - (x - 0.25) / 0.25 * 255
        g = 255
        r = 0
    elif x >= 0.5 and x < 0.75:
        b = 0
        g = 255
        r = (x - 0.5) / 0.25 * 255
    else:
        b = 0
        g = 255 - (x - 0.75) / 0.25 * 255
        r = 255
    return int(b), int(g), int(r)

def create_img(k, j, i, variable):
    ps = np.array([0,1,2,0,4,5,0,7,8,9,8,11,12,8,14,15])
    qs = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])
    xs = variable.data[j, 0, i, :17].copy()
    ys = variable.data[j, 0, i, 17:].copy()
    xs *= 100
    xs += 100
    ys *= -100
    ys += 150
    img = np.zeros((350, 200, 3), dtype=np.uint8) + 160
    img = cv2.line(img, (100, 0), (100, 350), (255, 255, 255), 1)
    img = cv2.line(img, (0, 150), (200, 150), (255, 255, 255), 1)
    np.random.seed(3 + row * k + j)
    rec_color = tuple(color_jet(np.random.uniform(0, 1)))
    img = cv2.rectangle(img, (0, 0), (200, 350), rec_color, 3)
    for i, (p, q) in enumerate(zip(ps, qs)):
        c = 1 / (len(ps) - 1) * i
        b, g, r = color_jet(c)
        img = cv2.line(img, (xs[p], ys[p]), (xs[q], ys[q]), (b, g, r), 2)
    for i in range(17):
        c = 1 / 16 * i
        b, g, r = color_jet(c)
        img = cv2.circle(img, (xs[i], ys[i]), 3, (b, g, r), 3)
    return img

In [47]:
col, row = 4, 4
l_seq = 64
imgs = np.zeros((l_seq, 350 * col, 600 * row, 3), dtype=np.uint8)
model = net.ConvAE(l_latent=64, l_seq=l_seq)
model_path = 'results/2017-12-05_15-36-37/gen_epoch_754.npz'
serializers.load_npz(model_path, model)
train = dataset.PoseDataset('data/h3.6m', length=l_seq, train=False)
train_iter = chainer.iterators.SerialIterator(train, batch_size=row, shuffle=True, repeat=False)
with chainer.no_backprop_mode(), chainer.using_config('train', False):
    pbar = ProgressBar(col)
    for k in range(col):
        batch = train_iter.next()
        batch = chainer.dataset.concat_examples(batch)
        xy, z = batch
        xy = Variable(xy)
        z_pred = model(xy)
        theta = np.array([np.pi / 2] * len(xy), dtype=np.float32)
        cos_theta = Variable(np.broadcast_to(np.cos(theta), z_pred.shape[::-1]).transpose(3, 2, 1, 0))
        sin_theta = Variable(np.broadcast_to(np.sin(theta), z_pred.shape[::-1]).transpose(3, 2, 1, 0))
        x = xy[:, :, :, :17]
        y = xy[:, :, :, 17:]
        real = F.concat((x * cos_theta + z * sin_theta, y), axis=3)
        fake = F.concat((x * cos_theta + z_pred * sin_theta, y), axis=3)
        
        for j in range(row):
            for i in range(l_seq):
                im0 = create_img(k, j, i, xy)
                im1 = create_img(k, j, i, real)
                im2 = create_img(k, j, i, fake)
                imgs[i, k*350:(k+1)*350, j*600:(j+1)*600] = np.concatenate((im0, im1, im2), axis=1)
        pbar.update(k + 1)

print('Saving video ...')
fourcc = cv2.VideoWriter_fourcc(*'H264')
if not os.path.exists(os.path.join(os.path.dirname(model_path), 'videos')):
    os.mkdir(os.path.join(os.path.dirname(model_path), 'videos'))
video_name = os.path.join(os.path.dirname(model_path), 'videos',
                          os.path.basename(model_path).replace('.npz', '.mp4'))
out = cv2.VideoWriter(video_name, fourcc, 20.0, (imgs.shape[2], imgs.shape[1]))
for img in imgs:
    out.write(img)
out.release()
print('Done!')

100% (4 of 4) |###########################| Elapsed Time: 0:00:12 ETA:  0:00:00

Saving video ...
Done!
